In [74]:
# These have to be installed through running "pip install -r requirements.txt"
import re
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # For stop words
nltk.download('wordnet')  # For lemmatization
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to C:\Users\Isakr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Isakr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Isakr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Isakr/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Storing the csv file as a pandas dataframe
df = pd.read_csv('Hotel_Reviews.csv')

df = df.sample(frac=0.0001, random_state=42)

In [76]:
def replace(cell):
    if cell == "No Positive" or cell == "No Negative":
        return ""
    return cell



df['Negative_Review'] = df['Negative_Review'].apply(replace)
df['Positive_Review'] = df["Positive_Review"].apply(replace)

In [77]:

def combine_names(row):
    return f"{row['Positive_Review']} {row['Negative_Review']}"


df["Review"] = df.apply(combine_names, axis="columns")

In [78]:
df = df.drop(["Hotel_Address", "Additional_Number_of_Scoring", "Review_Date", "Average_Score", "Reviewer_Nationality", "Review_Total_Negative_Word_Counts", "Review_Total_Positive_Word_Counts", "Total_Number_of_Reviews_Reviewer_Has_Given", "Reviewer_Score", "Tags", "days_since_review", "lat", "lng", "Hotel_Name", "Total_Number_of_Reviews", "Negative_Review", "Positive_Review"], axis = "columns")
df

,Review
488440,Hotel was great clean friendly staff free bre...
274649,No tissue paper box was present at the room
374688,Nice welcoming and service Pillows
404352,Everything including the nice upgrade The Hot...
451596,Lovely hotel v welcoming staff
...,...
2548,Love the biscuits which came with the tea n c...
147192,Loved the friendly staff The room had an air ...
158506,Overall it was excellent Our room was quite ...
357182,The reception staff and customer service chap...


In [79]:

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the preprocessing function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # Lemmatize the tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Join the tokens back into a single string
    processed_text = ' '.join(lemmatized_tokens)
    
    return processed_text

# Apply the preprocessing function to your DataFrame
df['Review'] = df['Review'].apply(preprocess_text)

# Display the DataFrame
print(df)

KeyboardInterrupt: 

['C:\\Users\\Isakr/nltk_data', 'c:\\Users\\Isakr\\AppData\\Local\\Programs\\Python\\Python311\\nltk_data', 'c:\\Users\\Isakr\\AppData\\Local\\Programs\\Python\\Python311\\share\\nltk_data', 'c:\\Users\\Isakr\\AppData\\Local\\Programs\\Python\\Python311\\lib\\nltk_data', 'C:\\Users\\Isakr\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']
